In [1]:
import numpy as np
import time
import pickle
import os
import matplotlib.pyplot as plt
import warnings
import pandas as pd

warnings.filterwarnings('ignore')

In [2]:
print("----------------Reading the Data-------------------------")
PATH = os.getcwd()
os.chdir('Alphabets/')

X_train = pd.read_csv('train.csv', sep=',', header=None, index_col=False)
X_test = pd.read_csv('test.csv', sep=',', header=None, index_col=False)
np.random.shuffle(X_train.to_numpy())
train_class = X_train[X_train.columns[-1]]
test_actual_class = X_test[X_test.columns[-1]]

X_train = X_train.drop(X_train.columns[-1], axis=1)
X_test = X_test.drop(X_test.columns[-1], axis=1)

print("----------------Data Reading completed-------------------")

os.chdir('../')

X_train = X_train/255
X_test = X_test/255
m = X_train.shape[0] # Number of Training Samples
n = X_train.shape[1] # Number of input features

print("The total number of training samples = {}".format(m))
print("The number of features = {}".format(n))

----------------Reading the Data-------------------------
----------------Data Reading completed-------------------
The total number of training samples = 13000
The number of features = 784


In [3]:
#To get the one hot encoding of each label
print("--------Perform 1-hot encoding of class labels------------")

train_class_enc = pd.get_dummies(train_class).to_numpy()
test_actual_class_enc = pd.get_dummies(test_actual_class).to_numpy()

--------Perform 1-hot encoding of class labels------------


In [4]:
#Add the intercept term to the data samples both in training and test dataset
X_train = np.hstack((np.ones((m,1)),X_train.to_numpy()))
X_test = np.hstack((np.ones((X_test.shape[0],1)),X_test.to_numpy()))

In [5]:
lr = 0.1
arch_test = [1,5,10,50,100]
arch = [50] #means one hidden layer with 2 perceptrons 
batch_size = 100 # Mini-Batch Size
r = np.max(train_class) + 1 # Default value of the number of classes = 26

In [6]:
#Mini-Batch formation
mini_batch = [(X_train[i:i+batch_size,:], train_class_enc[i:i+batch_size]) for i in range(0, m, batch_size)]
print("The number of mini-batches formed is = {}".format(len(mini_batch)))

The number of mini-batches formed is = 130


In [7]:
#Theta Initialization 
def theta_init(arch=50):
    theta = []
    arch = [arch]
    for i in range(len(arch)+1):
        if i == 0:
            dim0=n+1
            dim1=arch[i]
        elif (i == len(arch)):
            dim0=arch[i-1]
            dim1 = r
        else:
            dim0=arch[i-1]
            dim1= arch[i]

        theta.append(2*np.random.random((dim0, dim1))-1)
        #theta.append(np.zeros((dim0, dim1)))
    return theta

In [8]:
def activation(x):
    return 1/(1+np.exp(-x))

In [9]:
def forward_prop(data, theta):
    fm = []
    fm.append(data)
    for l in range(len(theta)):
        fm.append(activation(np.dot(fm[l], theta[l])))
    return fm

In [10]:
def cost_total(X, theta, Y, m):
    fm = forward_prop(X, theta)
    cost = (1/(2*m))*np.sum((Y-fm[-1])**2)
    return cost

In [11]:
def calc_accuracy(data, theta, actual_class):
    pred_class = forward_prop(data, theta)
    test_pred_class = pred_class[-1]
    for i in range(len(test_pred_class)):
        test_pred_class[i][test_pred_class[i] == np.max(test_pred_class[i])] = 1
        test_pred_class[i][test_pred_class[i] != np.max(test_pred_class[i])] = 0


    test_acc = 0
    for i in range(len(actual_class)):
        if (np.array_equal(test_pred_class[i], actual_class[i])):
            test_acc+=1
    test_acc /= data.shape[0]

    #print("The Test Accuracy of the model = {}%".format(test_acc*100))
    return (test_acc*100)

In [12]:
epochs = []
train_accuracy = []
test_accuracy = []
train_time = []

In [14]:
arch_test=[100]
lr0 = 0.5
ite = 1
for i in range(len(arch_test)):
    theta = theta_init(arch_test[i])
    fm = forward_prop(X_train, theta)
    
    epoch = 0
    start = time.time()
    cost_init = cost_total(X_train, theta, train_class_enc, m)

    while(True):
        count = 0

        print("Initial Cost for this epoch {} = {}".format(epoch, cost_init))

        for b in mini_batch:
            lr = lr0/ite
            #print("The learning rate = {}".format(lr))
            X_b = b[0]
            Y_b = b[1]
            fm = forward_prop(X_b, theta)
            delta = [None]*len(fm)

            if (count % 60 == 0):
                print("Error = "+str(cost_total(X_b, theta, Y_b, batch_size)))
            #Backward Propagation

            for l in range(len(fm)-1, 0, -1):
                if (l == len(fm)-1):
                    delta[l] = ((1/batch_size)*(Y_b - fm[l])*fm[l]*(1-fm[l]))
                else:
                    delta[l]=(np.dot(delta[l+1], theta[l].T)*fm[l]*(1-fm[l]))

            #delta_l1 = np.dot(delta_l2, theta2.T)*l1*(1-l1)

            for t in range(len(theta)):
                theta[t] += lr*np.dot(fm[t].T, delta[t+1]) 
            ite+=1
            count+=1
        epoch+=1 #Number of epochs
        cost_final = cost_total(X_train, theta, train_class_enc, m)
        print("Cost after {} epochs is = {}".format(epoch, cost_final))
        if (abs(cost_final-cost_init) < 1e-06):
            print("cost initial= {} , cost final={} , change in cost= {}".format(cost_init,cost_final, cost_final-cost_init))
            break
        cost_init = cost_final
    epochs.append(epoch)
    train_time.append(time.time()-start)
    train_accuracy.append(calc_accuracy(X_train, theta, train_class_enc))
    test_accuracy.append(calc_accuracy(X_test, theta, test_actual_class_enc))
    print("\n------------------------------------------------------------------------------")
    print("The stats for number of units in the hidden layer = {} are as below:".format(arch_test[i]))
    print("------------------------------------------------------------------------------")
    print("The number of epochs = {}".format(epochs[-1]))
    print("The training time = {}sec".format(train_time[-1]))
    print("The training accuracy is = {}%".format(train_accuracy[-1]))
    print("The test accuracy is = {}%".format(test_accuracy[-1]))
    print("------------------------------------------------------------------------------\n")

Initial Cost for this epoch 0 = 4.923117292813985
Error = 5.007692282731191
Error = 1.4308962519293544
Error = 1.1707670092058968
Cost after 1 epochs is = 1.1580230690078994
Initial Cost for this epoch 1 = 1.1580230690078994
Error = 1.1638416348574776
Error = 1.0734766821989734
Error = 0.9808869666142325
Cost after 2 epochs is = 0.9785939932898726
Initial Cost for this epoch 2 = 0.9785939932898726
Error = 0.991020167873485
Error = 0.9611568639919809
Error = 0.8969507899191362
Cost after 3 epochs is = 0.8970878692543235
Initial Cost for this epoch 3 = 0.8970878692543235
Error = 0.9097073193575697
Error = 0.9006582459774112
Error = 0.8471350552106713
Cost after 4 epochs is = 0.8489068325279998
Initial Cost for this epoch 4 = 0.8489068325279998
Error = 0.8606202539682107
Error = 0.8613739425585177
Error = 0.8136794070265075
Cost after 5 epochs is = 0.8165790074403896
Initial Cost for this epoch 5 = 0.8165790074403896
Error = 0.8273653658026278
Error = 0.8332461105173703
Error = 0.78951306

Error = 0.6460142068111943
Cost after 47 epochs is = 0.6471932334382139
Initial Cost for this epoch 47 = 0.6471932334382139
Error = 0.6529003542360953
Error = 0.6579673142669432
Error = 0.6452534588100496
Cost after 48 epochs is = 0.6463781550854216
Initial Cost for this epoch 48 = 0.6463781550854216
Error = 0.6520596129361613
Error = 0.6570334422318059
Error = 0.6445158946463764
Cost after 49 epochs is = 0.6455876210422087
Initial Cost for this epoch 49 = 0.6455876210422087
Error = 0.6512439470008243
Error = 0.6561275410593344
Error = 0.6438003208148393
Cost after 50 epochs is = 0.6448203844557834
Initial Cost for this epoch 50 = 0.6448203844557834
Error = 0.6504520859348881
Error = 0.6552482103511913
Error = 0.6431056287012767
Cost after 51 epochs is = 0.6440752860067303
Initial Cost for this epoch 51 = 0.6440752860067303
Error = 0.649682848497543
Error = 0.6543941462152966
Error = 0.6424307869643313
Cost after 52 epochs is = 0.6433512461506957
Initial Cost for this epoch 52 = 0.6433

Error = 0.6317691988167166
Error = 0.6244778734715353
Cost after 94 epochs is = 0.6240231285961432
Initial Cost for this epoch 94 = 0.6240231285961432
Error = 0.628880878481665
Error = 0.6314175531177147
Error = 0.6241971432973772
Cost after 95 epochs is = 0.6237201590930208
Initial Cost for this epoch 95 = 0.6237201590930208
Error = 0.6285648748539578
Error = 0.63107118773115
Error = 0.6239205615274583
Cost after 96 epochs is = 0.6234216531783209
Initial Cost for this epoch 96 = 0.6234216531783209
Error = 0.6282534716517646
Error = 0.6307299666754836
Error = 0.6236480218806264
Cost after 97 epochs is = 0.6231274971314098
Initial Cost for this epoch 97 = 0.6231274971314098
Error = 0.6279465524783115
Error = 0.630393758870438
Error = 0.6233794219058929
Cost after 98 epochs is = 0.6228375813037077
Initial Cost for this epoch 98 = 0.6228375813037077
Error = 0.6276440050881225
Error = 0.6300624379114418
Error = 0.623114662805692
Cost after 99 epochs is = 0.6225517999320468
Initial Cost for

Cost after 140 epochs is = 0.6134594373868535
Initial Cost for this epoch 140 = 0.6134594373868535
Error = 0.617828287179564
Error = 0.6193706442037897
Error = 0.6145333081522474
Cost after 141 epochs is = 0.6132852496327245
Initial Cost for this epoch 141 = 0.6132852496327245
Error = 0.6176454272970955
Error = 0.6191725990125653
Error = 0.6143736002272299
Cost after 142 epochs is = 0.6131127596633029
Initial Cost for this epoch 142 = 0.6131127596633029
Error = 0.6174643298119046
Error = 0.6189765059817708
Error = 0.6142154371565176
Cost after 143 epochs is = 0.6129419385600776
Initial Cost for this epoch 143 = 0.6129419385600776
Error = 0.6172849649320233
Error = 0.618782331715491
Error = 0.6140587927496333
Cost after 144 epochs is = 0.6127727581015059
Initial Cost for this epoch 144 = 0.6127727581015059
Error = 0.617107303579752
Error = 0.6185900436233669
Error = 0.6139036414450897
Cost after 145 epochs is = 0.6126051907413963
Initial Cost for this epoch 145 = 0.6126051907413963
Erro

Cost after 186 epochs is = 0.6068552259672737
Initial Cost for this epoch 186 = 0.6068552259672737
Error = 0.6108810858970652
Error = 0.6118783776800927
Error = 0.6084695506174341
Cost after 187 epochs is = 0.606737115162218
Initial Cost for this epoch 187 = 0.606737115162218
Error = 0.6107565755558814
Error = 0.6117447111026068
Error = 0.6083609424303172
Cost after 188 epochs is = 0.6066198616712144
Initial Cost for this epoch 188 = 0.6066198616712144
Error = 0.6106329597850122
Error = 0.6116120267190834
Error = 0.6082531167935664
Cost after 189 epochs is = 0.6065034545508456
Initial Cost for this epoch 189 = 0.6065034545508456
Error = 0.6105102272504348
Error = 0.6114803119157547
Error = 0.6081460637628008
Cost after 190 epochs is = 0.6063878830561161
Initial Cost for this epoch 190 = 0.6063878830561161
Error = 0.6103883668224933
Error = 0.611349554308232
Error = 0.6080397735733497
Cost after 191 epochs is = 0.606273136635809
Initial Cost for this epoch 191 = 0.606273136635809
Error 

...

Error = 0.5945112354712137
Error = 0.594214747396944
Cost after 416 epochs is = 0.5913697082179451
Initial Cost for this epoch 416 = 0.5913697082179451
Error = 0.5944803200080814
Error = 0.594466114844249
Error = 0.5941773063730876
Cost after 417 epochs is = 0.5913294267708337
Initial Cost for this epoch 417 = 0.5913294267708337
Error = 0.5944374707036447
Error = 0.5944211385927739
Error = 0.5941399829135046
Cost after 418 epochs is = 0.5912892723531434
Initial Cost for this epoch 418 = 0.5912892723531434
Error = 0.5943947556568394
Error = 0.5943763058966259
Error = 0.5941027763580602
Cost after 419 epochs is = 0.5912492442504267
Initial Cost for this epoch 419 = 0.5912492442504267
Error = 0.5943521741168425
Error = 0.594331615942301
Error = 0.5940656860519341
Cost after 420 epochs is = 0.5912093417539882
Initial Cost for this epoch 420 = 0.5912093417539882
Error = 0.5943097253388488
Error = 0.5942870679228804
Error = 0.5940287113455635
Cost after 421 epochs is = 0.5911695641608282
Ini

Error = 0.5925736632229597
Error = 0.5926049155617937
Cost after 462 epochs is = 0.589638254907915
Initial Cost for this epoch 462 = 0.589638254907915
Error = 0.5926377141875017
Error = 0.5925344257381437
Error = 0.5925722717338114
Cost after 463 epochs is = 0.5896031558446551
Initial Cost for this epoch 463 = 0.5896031558446551
Error = 0.5926003457518259
Error = 0.5924953008581875
Error = 0.5925397198631471
Cost after 464 epochs is = 0.5895681560997996
Initial Cost for this epoch 464 = 0.5895681560997996
Error = 0.5925630824290041
Error = 0.5924562880083926
Error = 0.5925072594858705
Cost after 465 epochs is = 0.5895332551713622
Initial Cost for this epoch 465 = 0.5895332551713622
Error = 0.5925259236906132
Error = 0.5924173866182547
Error = 0.5924748901414089
Cost after 466 epochs is = 0.5894984525609924
Initial Cost for this epoch 466 = 0.5894984525609924
Error = 0.5924888690120395
Error = 0.5923785961214247
Error = 0.5924426113725164
Cost after 467 epochs is = 0.5894637477739421
In

Error = 0.5911919406635262
Cost after 508 epochs is = 0.5881194222834686
Initial Cost for this epoch 508 = 0.5881194222834686
Error = 0.5910201128370123
Error = 0.590842600341918
Error = 0.591163092979466
Cost after 509 epochs is = 0.5880884224188013
Initial Cost for this epoch 509 = 0.5880884224188013
Error = 0.5909870851978853
Error = 0.5908080956092634
Error = 0.5911343189611847
Cost after 510 epochs is = 0.5880575020829477
Initial Cost for this epoch 510 = 0.5880575020829477
Error = 0.5909541418255377
Error = 0.5907736804342723
Error = 0.5911056182713399
Cost after 511 epochs is = 0.5880266609109908
Initial Cost for this epoch 511 = 0.5880266609109908
Error = 0.5909212823352787
Error = 0.5907393544029347
Error = 0.5910769905748079
Cost after 512 epochs is = 0.5879958985404153
Initial Cost for this epoch 512 = 0.5879958985404153
Error = 0.5908885063449385
Error = 0.5907051171039792
Error = 0.5910484355386629
Cost after 513 epochs is = 0.5879652146110865
Initial Cost for this epoch 5

Error = 0.5893730559161411
Error = 0.5899364080917001
Cost after 554 epochs is = 0.5867705669877437
Initial Cost for this epoch 554 = 0.5867705669877437
Error = 0.5895826017569274
Error = 0.5893422070508827
Error = 0.5899106307083386
Cost after 555 epochs is = 0.5867428811326693
Initial Cost for this epoch 555 = 0.5867428811326693
Error = 0.5895530872727739
Error = 0.5893114315242366
Error = 0.5898849135100384
Cost after 556 epochs is = 0.58671526022569
Initial Cost for this epoch 556 = 0.58671526022569
Error = 0.5895236416753643
Error = 0.5892807290252704


KeyboardInterrupt: 

In [16]:
calc_accuracy(X_test, theta, test_actual_class_enc)

3.8461538461538463